In [3]:
import sqlite3
import pandas as pd
import os
from joblib import Parallel, delayed

In [4]:
##################################
# Python SQLite Conversion Reader
##################################
last_db_path = None
sql_columns = []

class FromSQLite:
    pathDB = os.path.join(os.getcwd(), "basedata", "total_base_data.db")
    
    @staticmethod
    def infer_sql_type(dtype):
        if pd.api.types.is_integer_dtype(dtype):
            return "INTEGER"
        elif pd.api.types.is_float_dtype(dtype):
            return "REAL"
        elif pd.api.types.is_datetime64_any_dtype(dtype):
            return "DATETIME"
        else:
            return "TEXT"  # Default to TEXT for other types

    @staticmethod
    def getData(SQL_columns='*', importer=None, exporter=None, year=None, product_code=None, value=None, quantity=None, table_name="base_data", path=pathDB):
        """
        Fetch data from SQLite with optional filters.

        Args:
            SQL_columns (str): Columns to select. Default is '*'.
            importer (list, optional): List of import countries. Default is None.
            exporter (list, optional): List of export countries. Default is None.
            year (list, optional): List of years. Default is None.
            product_code (list, optional): List of product codes. Default is None.
            value (float, optional): Minimum value. Default is None.
            quantity (float, optional): Minimum quantity. Default is None.
            table_name (str, optional): Name of the table in the database. Default is "base_data".
            path (str, optional): Path to the database file. Default is pathDB.

        Returns:
            pd.DataFrame: Filtered data.
        """
        if not os.path.exists(path):
            print(f"ERROR: Database file '{path}' does not exist.")
            return None

        query = f"SELECT {SQL_columns} FROM {table_name} WHERE 1=1"
        params = []
        
        if exporter:
            print('Getting exporter', end="")
            placeholders = ', '.join('?' for _ in exporter)
            query += f" AND export_country IN ({placeholders})"
            params.extend(exporter)

        if importer:
            print('Getting importer', end="")
            placeholders = ', '.join('?' for _ in importer)
            query += f" AND import_country IN ({placeholders})"
            params.extend(importer)

        if year:
            print('Getting years', end="")
            placeholders = ', '.join('?' for _ in year)
            query += f" AND year IN ({placeholders})"
            params.extend(year)

        if product_code:
            print('Getting product codes', end="")
            placeholders = ', '.join('?' for _ in product_code)
            query += f" AND code IN ({placeholders})"
            params.extend(product_code)

        if value:
            print('Getting value', end="")
            placeholders = ', '.join('?' for _ in value)
            query += f" AND value IN ({placeholders})"
            params.append(value)

        if quantity:
            print('Getting quantity', end="")
            placeholders = ', '.join('?' for _ in quantity)
            query += f" AND quantity IN ({placeholders})"
            params.append(quantity)

        try:
            with sqlite3.connect(path) as conn:
                df = pd.read_sql_query(query, conn, params=params)
        except sqlite3.OperationalError as e:
            print(f"SQL Error: {e}")
            return None

        return df

    def pushData(df, table_name='base_data', db_path=pathDB):
        """
        Push data into SQLite database, adding only new rows and handling missing columns.

        Args:
            df (pd.DataFrame): Data to push into the database.
            table_name (str): Target table name.
            db_path (str): Path to the SQLite database file.
        """
        global last_db_path, sql_columns  # Declare global variables

        db_path = db_path or FromSQLite.pathDB

        # Ensure the directory for the database exists
        if not os.path.exists(os.path.dirname(db_path)):
            os.makedirs(os.path.dirname(db_path))

        try:
            # Connect to the SQLite database
            with sqlite3.connect(db_path) as conn:
                cursor = conn.cursor()

                # Check if the table exists
                cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';")
                table_exists = cursor.fetchone() is not None

                if not table_exists:
                    # Create the table with the DataFrame's schema
                    print(f"Table '{table_name}' does not exist. Creating it...")
                    df.head(0).to_sql(table_name, conn, if_exists='replace', index=False)

                # Check if the database path is the same as the last call
                if db_path != last_db_path:
                    # Cache the SQL column names for the table
                    cursor.execute(f"PRAGMA table_info({table_name});")
                    sql_columns = [row[1] for row in cursor.fetchall()]  # Get column names
                    last_db_path = db_path

                # Check for missing columns
                df_columns = df.columns.tolist()
                print(df_columns)
                missing_in_sql = set(df_columns) - set(sql_columns)

                # Add missing columns to the table if any
                if missing_in_sql:
                    for column in missing_in_sql:
                        col_type = FromSQLite.infer_sql_type(df[column].dtype)
                        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column} {col_type};")
                    print(f"Added missing columns: {missing_in_sql}", end="")

                # Push data into the table
                # Use pandas' `to_sql` with `if_exists='append'` to handle insertion
                df.to_sql(table_name, conn, if_exists='append', index=False)

                print(f"Data successfully pushed to table '{table_name}'.", end="")

        except Exception as e:
            print(f"Error pushing data: {e}")

    @staticmethod
    def updateData(table_name, updates, conditions, db_path=None):
        """
        Update data in SQLite database based on conditions.
        """
        db_path = db_path or os.path.join(os.getcwd(), "basedata", "total_base_data.db")

        try:
            with sqlite3.connect(db_path) as conn:
                cursor = conn.cursor()
                update_clause = ', '.join(f"{k} = ?" for k in updates.keys())
                condition_clause = ' AND '.join(f"{k} = ?" for k in conditions.keys())
                query = f"UPDATE {table_name} SET {update_clause} WHERE {condition_clause}"
                params = list(updates.values()) + list(conditions.values())
                cursor.execute(query, params)
                conn.commit()
        except Exception as e:
            print(f"Error updating data: {e}")

    def summarize(db_path=pathDB):
        # Get database size
        db_size = os.path.getsize(db_path) / (1024 * 1024)
        if db_size > 1000: 
            db_size /= 1024
            print(f"Database size: {db_size:.2f} GB\n")
        else:
            print(f"Database size: {db_size:.2f} MB\n")

        # Connect to the database
        with sqlite3.connect(db_path) as conn:
            cursor = conn.cursor()

            # List all tables
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = [row[0] for row in cursor.fetchall()]
            print("Tables in the database:")
            print(tables, "\n")

            # Summarize each table
            for table in tables:
                print(f"Summary for table '{table}':")

                # Get schema
                cursor.execute(f"PRAGMA table_info({table});")
                schema = cursor.fetchall()
                print("Schema:")
                for column in schema:
                    print(f"  {column}")

                # Get row count
                cursor.execute(f"SELECT COUNT(*) FROM {table};")
                row_count = cursor.fetchone()[0]
                print(f"Row count: {row_count}")

                # Get sample data
                try:
                    # Random sample (5 rows)
                    random_sample = pd.read_sql_query(f"SELECT * FROM {table} ORDER BY RANDOM() LIMIT 5;", conn)
                    print("\nRandom Sample:")
                    print(random_sample)

                    # First 5 rows sorted by year
                    first_by_year = pd.read_sql_query(f"SELECT * FROM {table} ORDER BY year ASC LIMIT 5;", conn)
                    print("\nFirst 5 Rows by Year:")
                    print(first_by_year)

                    # Last 5 rows sorted by year
                    last_by_year = pd.read_sql_query(f"SELECT * FROM {table} ORDER BY year DESC LIMIT 5;", conn)
                    print("\nLast 5 Rows by Year:")
                    print(last_by_year)

                except Exception as e:
                    print(f"Error fetching sample data: {e}")

                print("\n")


In [ ]:
def whole_file_processing(file, product_codes, country_codes, test = None):
    file_path = os.path.join(os.getcwd(), "dataset", file)
    df = pd.read_csv(file_path, low_memory=False)

    df["k"] = df["k"].astype(str)  # Convert to string
    product_codes["code"] = product_codes["code"].astype(str) 
    # First merge for column 'i'
    df = pd.merge(
        left=df,
        right=country_codes.rename(columns={"country_code": "country_code_i"}),
        left_on="i",
        right_on="country_code_i",
        how="left"
    ).rename(columns={"country_name": "export_country", "country_iso3": "export_country_iso3"})

    df = pd.merge(
        left=df,
        right=country_codes.rename(columns={"country_code": "country_code_j"}),
        left_on="j",
        right_on="country_code_j",
        how="left"
    ).rename(columns={"country_name": "import_country", "country_iso3": "import_country_iso3", "t": "year", "k":"code", "v":"value", "q":"quantity"}).drop(
        columns=["country_iso2_x", "country_iso2_y", "country_code_i", "country_code_j", "i", "j","export_country_iso3","import_country_iso3"], axis=1
    )

    if df is None:
        print(f"Skipping file: {file}")
        return None
    
    if test:
        return df
    else:
        FromSQLite.pushData(df)
        print(f'{file} succesfully pushed to SQLite', end="")

In [ ]:
#################################
# SQLite Import/Export Processing
#################################
pathDB = os.path.join(os.getcwd(), "basedata", "total_base_data.db")
if os.path.exists(pathDB):
    print("Loading precomputed file")
    # df = FromSQLite.getData()
    
else:
    files_to_process = [
        file for file in os.listdir(os.path.join(os.getcwd(), "dataset"))
        if file.startswith("BACI_") and file.endswith(".csv")
    ]
    print(files_to_process, end="")

    country_codes = pd.read_csv(os.path.join("dataset", "country_codes_V202401b.csv"))
    product_codes = pd.read_csv(os.path.join("dataset", "product_codes_HS22_V202401b.csv"))

    Parallel(n_jobs=1, backend='loky')(
        delayed(lambda file : FromSQLite.pushData(whole_file_processing(file, product_codes, country_codes), table_name="base_data", db_path=pathDB))
        (file) for file in files_to_process
    )

Loading precomputed file


In [6]:
###########################
# SQLite Gravity Processing
###########################
pathDB = os.path.join(os.getcwd(), "basedata", "gravity.db")
if os.path.exists(pathDB):
    print("Loading precomputed file")
    # df = FromSQLite.getData()
    
else:
    """
    files_to_process = [
        file for file in os.listdir(os.path.join(os.getcwd(), "gravity"))
        if file.startswith("BACI_") and file.endswith(".csv")
    ]
    """
    gravity_path = os.path.join(os.getcwd(), "gravity", "Gravity_V202211.csv")
    gravity_df = pd.read_csv(gravity_path)

    FromSQLite.pushData(gravity_df, db_path=pathDB)


Loading precomputed file


In [ ]:
##################################
# Cleanup the mess of a SQL table
##################################

pathDB = os.path.join(os.getcwd(), "basedata", "total_base_data.db")
conn = sqlite3.connect(pathDB)
cursor = conn.cursor()
# Execute the update query
cursor.execute("""
    UPDATE base_data
    SET code = printf('%06d', code)
    WHERE LENGTH(code) < 6;
""")

# Commit changes and close the connection
conn.commit()
conn.close()

In [4]:
pathDB = os.path.join(os.getcwd(), "basedata", "total_base_data.db")
FromSQLite.summarize(pathDB)

Database size: 12.71 GB

Tables in the database:
['base_data'] 

Summary for table 'base_data':
Schema:
  (0, 'year', 'INT', 0, None, 0)
  (1, 'code', 'TEXT', 0, None, 0)
  (2, 'value', 'REAL', 0, None, 0)
  (3, 'quantity', 'REAL', 0, None, 0)
  (4, 'export_country', 'TEXT', 0, None, 0)
  (5, 'import_country', 'TEXT', 0, None, 0)
Row count: 247174310

Random Sample:
   year    code   value  quantity export_country import_country
0  2003  701890  43.889     1.500         Israel        Austria
1  2021  480441   1.626     0.129          China         France
2  2002  220190   3.292     8.889      Australia          China
3  2011  521132   0.161     0.025     Bangladesh        Czechia
4  2007  560900   6.596     2.502     Costa Rica      Nicaragua

First 5 Rows by Year:
   year    code   value  quantity export_country import_country
0  1995  841510  36.687     5.812    Afghanistan        Algeria
1  1995  570110  11.060     0.195    Afghanistan        Andorra
2  1995  080620  11.804    15.00

In [3]:
##################################
# Aggregate Category
##################################
pathDB = os.path.join(os.getcwd(), "basedata", "total_base_data.db")
query = """
SELECT
    year,
    import_country,
    export_country,
    SUBSTR('00000' || CAST(code AS TEXT), -6, 2) AS code,
    SUM(value) AS value,
    SUM(quantity) AS quantity,
    COUNT(DISTINCT code) AS distinct_products
FROM
    base_data
GROUP BY
    year, import_country, export_country, code
ORDER BY
    year;
"""
with sqlite3.connect(pathDB) as conn:
    df = pd.read_sql_query(query, conn)

print(df.head())


: 

In [ ]:
FromSQLite.pushData(df, table_name='aggregate 1')

In [ ]:
###########################
# Database cleanup
############################ 
# Connect to the database
conn = sqlite3.connect(pathDB)
cursor = conn.cursor()

# Add a new column
cursor.execute("ALTER TABLE base_data ADD COLUMN value_real REAL")

# Update the new column with converted values
cursor.execute("UPDATE base_data SET value_real = CAST(value AS REAL)")

# Rename the column (workaround for SQLite's limitation)
cursor.execute("""
    CREATE TABLE base_data_new AS
    SELECT 
        year,
        code,
        value,
        quantity_real AS quantity,
        export_country,
        import_country
    FROM base_data
""")

# Drop the old table
cursor.execute("DROP TABLE base_data")

# Rename the new table
cursor.execute("ALTER TABLE base_data_new RENAME TO base_data")

# Commit and close
conn.commit()
conn.close()
